## Metaprogramming

In [1]:
## One of the most important mantras of software development is dont repeat yourself. That is nay time you are faced with a problem of creating highly repetitive code it often pays to look for a more elegant solution. In a nutshell metaprogramming is about creating functions and classes whose main goal is to manipulate code. the main features for this include decorators class decorators and metaclasses. However a variety of other useful topics including signature objects execution of code with exec() and inspecting the internals of funcitons and lcasses enter the picture. tThe main purpose of this chapter is to explore varous metaprogramming techniques and to give examples of how they can be used to customize the behavior of Python to your own whims.




## Putting a Wrapper Around a Function

In [6]:
#First create a wrapper using @wraps

import time
from functools import wraps

def timethis(func):
    ''' 
    Decorator that reports the execution time
    '''

    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [7]:
#Here is an exaple of using the decorator

@timethis
def countdown(n):
    while n>0:
        n -= 1

countdown(1000000)


countdown 0.0809779167175293


In [8]:
#decorator is a function that accepts a funciton as input and returns a new functon as ooutput. Whenever you write a code like this

@timethis
def countdown(n):
    pass

#its the same as if you had performed these separate steps:
def countdown(n):
    pass

countdown = timethis(countdown)

In [36]:
class A:
    @staticmethod
    def method(cls):
        cls.myvar = 1
        return cls.myvar

class B:
    def method(cls):
        pass
    method = classmethod(method)

In [41]:
c = A()
c.method(c)

c.myvar

1

## More examples of classmethods and staticmethos

In [53]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    @classmethod
    def from_birth_year(cls, birth_year):
        current_year = 2023
        age = current_year - birth_year
        return cls(name='Unknown', age = age)


p = Person('Arka', 31) #This is the generic initialization method where you supply all the necessary data

In [50]:
#However if you dont have the age data, but you have birth year and you want to create the instance of a class from that data, you can use the classmethod to do that like the following example
p = Person.from_birth_year(1992)

In [52]:
# p  is initialized with all the necessary data
p.name

'Unknown'

In [55]:
#example of staticmethod

class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    @classmethod
    def from_birth_year(cls, birth_year):
        current_year = 2023
        age = current_year - birth_year
        return cls(name='Unknown', age = age)
    
    @staticmethod
    def is_adult(age):
        return age>18
    

p = Person.from_birth_year(1992)
p.is_adult(32)  # Doesnot depend on the data of the instance

True

# Preserving Function Metadata 2hen writing decorators

In [56]:
import time
from functools import wraps


def timethis(func):
    '''
    Decorator that reports the execution time.
    '''

    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'{func.__name__} executed in {end - start} seconds')
        return result

    return wrapper


@timethis
def countdown(n):
    '''
    Counts down from n to 0
    '''
    while n>1:
        n -= 1
        

In [59]:
countdown(10)
countdown.__name__
countdown.__doc__

countdown executed in 3.814697265625e-06 seconds


'\n    Counts down from n to 0\n    '

## Unwrapping the decorator

In [65]:
countdown.__wrapped__(10)
 

## Defining a decorator that takes arguments

In [67]:
from functools import wraps
import logging


def logged(level, name=None, message=None):
    '''
    Add logging to a function. Level is the logging level,
    name is the logger name, and message is the log message.
    If name and message arent specified, They default to the function's
    module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate


#Example usecase
@logged(logging.DEBUG)
def add(x,y):
    return x+y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

In [71]:
spam()

spam


Spam!


## Defining a decorator that takes and optional argument

In [78]:
from functools import wraps, partial
import logging



def logged(func=None, *, level=logging.DEBUG,name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)

    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__

    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        return func(*args, **kwargs)
    return wrapper

#Example use

@logged      #optional
def add(x,y):
    return x+y



@timethis  #additonal wrapper
@logged(level=logging.DEBUG, name='example')  #supplied
def spam():
    return 'Spam!'

In [77]:
spam()

spam executed in 9.059906005859375e-06 seconds


'Spam!'

## Defining decorators as part of the class

In [87]:
#You want to declare decorators as part of the class


from functools import wraps

class A:
    def decorator1(self,func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('decorator1')
            return func(*args, **kwargs)
        return wrapper
    

    @classmethod
    def decorator2(cls, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('decorator2')
            return func(*args, **kwargs)
        return wrapper
    


#Here is an example of using that decorator with classmethod
a = A.decorator2(add)
a(1,2)


#Otherwise the general method is
a = A()

@a.decorator1
def add(x,y):
    return x + y


#If we look carefully we will notice that one is applied from an instance a and the other is applied from the class A

decorator2
decorator1


In [90]:
#The built in property decorator is also a class with decorators


class Person:
    first_name = property()

    @first_name.getter
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value,str):
            raise TypeError("Expected a String")
        self._first_name = value

## Defining Decorators as Classes

In [7]:
## You wnat to wrap functions with decorators as classes


import types

from functools import wraps


class Profiled:
    def __init__(self, func):
        wraps(func)(self)
        self.ncalls = 0

    def __call__(self, *args, **kwargs):
         self.ncalls += 1
         return self.__wrapped__(*args, **kwargs)
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)


@Profiled
def add(x, y):
    return x + y

print(add(2, 3))
print(add.ncalls)

# The decorator



class Spam:
    @Profiled
    def bar(self, x):
        print(self, x)

5
1


In [8]:
s = Spam()
s.bar(1)

<__main__.Spam object at 0x7fa9a01180a0> 1


## Writing Decorators That Add Arguments to Wrapped Functions

In [9]:
from functools import wraps

def optional_debug(func):
    @wraps(func)   #---wrap the function
    def wrapper(*args, debug=False, **kwargs):  #---add optional argument within the function
        if debug:
            print('Calling', func.__name__) #-----perform action if the condition fulfills
        return func(*args, **kwargs)
    return wrapper

In [13]:
#How this helps

def a(x, debug=False):
    if debug:
        print("Calling a")

def b(x,y,z, debug = False):
    if debug:
        print("Calling b")

def c(x,y,debug=False):
    if debug:
        print("calling c")


# instead we do this

@optional_debug
def a(x):
    pass



In [17]:
a(12,debug=True)
b(12,11,19,debug=True)
c(3,4,debug=True)

Calling a
Calling b
calling c


## Using Decorators to Patch Class Definitions

In [21]:
#implementation

def log_getattribute(cls):
    # Get the original implementation

    orig_implementation = cls.__getattribute__

    # Make the new implementation
    def new_getatttribute(self, name):
        print("getting:",name)
        return orig_implementation(self, name)

    # Attach to the class and return
    cls.__getattribute__ = new_getatttribute
    return cls


# Example
@log_getattribute
class A:
    def __init__(self):
        self.x = 10
        self.y = 20

    def __getattribute__(self, name):
        print("getting:", name)
        return object.__getattribute__(self, name)




In [23]:
a = A()
a.x  ## Here it prints two times because it uses both the getattribute methods

getting: x
getting: x


10

In [25]:
#Conventional implementation

from typing import Any


class LoggedGetattribute:
    def __getattribute__(self, name):
        print('getting:', name)
        return super().__getattribute__(name)


#Example
class A(LoggedGetattribute):
    def __init__(self,x):
        self.x = x

    def spam(self):
        pass

In [31]:
class Spam1:
    def __init__(self):
        print(self.__class__,"Initiated")

#However you can customize this with __call__ func

class Spam2():
    def __call__(self, *args,**kwargs):
        raise TypeError("Cant instantiate directly")

# Example

class Spam(metaclass=NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok')


NameError: name 'NoInstances' is not defined

In [29]:
a = Spam1()

<class '__main__.Spam'> Initiated


## Enforcing an Argument Signature on *args and **kwargs

In [1]:
from inspect import Signature, Parameter

parms = [Parameter('x', Parameter.POSITIONAL_OR_KEYWORD),
         Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42),
         Parameter('z', Parameter.KEYWORD_ONLY, default=None)]

In [2]:
sig = Signature(parms)
print(sig)


(x, y=42, *, z=None)


In [3]:
def func(*args, **kwargs):
    bound_values = sig.bind(*args, **kwargs)
    for name, value in bound_values.arguments.items():
        print(name, value)

## Enforcing Coding Conventions in Classes

In [4]:
#Your program consists of a large class hierarchy and you would like to enforce certain
#kinds of coding conventions (or perform diagnostics) to help maintain programmer
#sanity.


# If you want to monitor the definition of classes you can ofter do it by defining a metaclass. A basic metaclass is usually define dby inheriting from type and redefining its __new__() method or __init__() method.


class MyMeta(type):
    def __new__(self, clsname, bases, clsdict):
        #clasname is the name of class beign defined
        #basese is tuple of base classe
        #clsdict is class dictionary

        return super().__new__(cls, clsname, bases, clsdict)

# Alternatively using __init__()

class MyMeta(type):
    def __init__(self, clsname, bases, clsdict):
        super().__init__(clsname, bases, clsdict)



# Example 
class Root(metaclass=MyMeta):
    pass

class A(Root):
    pass


class B(Root):
    pass


In [5]:
import collections

Stock = collections.namedtuple('Stock', ['name','shares','price'])

In [8]:
a = Stock('GOOG',2, '556')

In [13]:
#How to create the named tuples

import operator
import types
import sys


def named_tuple(classname, fieldnames):
    #Populate a dictionary of field property accessors

    cls_dict = {name: property(operator.itemgetter(n)) for n, name in enumerate(fieldnames)}

    # Make a __new__ function and add to the class dict

    def __new__(cls, *args):
        if len(args) != len(fieldnames):
            raise TypeError('Expected {} arguments'.formate(len(fieldnames)))
        return tuple.__new__(cls, args)
    
    cls_dict['__new__'] = __new__


    # Make the class

    cls = types.new_class(classname, (tuple,), {}, lambda ns: ns.update(cls_dict))

    #

In [15]:
# Performing initialization or setup actions at the time of class definition is a classic use of metaclasses..

import operator

class StructTupleMeta(type):
    def __init__(cls, *args, **kwargs):
        super().__init__(*args, **kwargs)
        for n, name in enumerate(cls._fields):
            setattr(cls, name, property(operator.itemgetter(n)))


class StructTuple(tuple, metaclass=StructTupleMeta):
    _fields = []
    def __new__(cls, *args):
        if len(args) !=  len(cls._fields):
            raise ValueError('{} argumetns required'.format(len(cls._fields)))
        return super().__new__(cls,args)

In [16]:
class Stock(StructTuple):
    _fields = ['name', 'shares', 'price']


class Point(StructTuple):
    _fields = ['x', 'y']


In [18]:
s = Stock('Acme', 50, 40)
s.name

'Acme'

## Defining Context Managers the Easy Way

In [3]:
import time
from contextlib import contextmanager

@contextmanager
def timethis(label):
    start = time.time()
    try:
        yield
    finally:
        end = time.time()
        print('{}: {}'.format(label, end - start))


In [4]:
with timethis('counting'):
    n = 100000
    while n>0:
        n -= 1

counting: 0.02538752555847168


In [5]:
@contextmanager
def list_transaction(orig_list):
    working = list(orig_list)
    yield working
    orig_list[:] = working

In [7]:
# Normally t write a context manager you define a class with an __enter__() and __exit__() emthod, like this:
import time
class timethis:
    def __init__(self, label):
        self.label = label
    def __enter__(self):
        self.start = time.time()

    def __exit__(self):
        self.stop = time.time()

## Executing Code with Local Side Effects

In [9]:
a = 13
exec("b  = a + 1")
print(b) # works

# But if you try it within a funciton

def test():
    a = 15
    exec('b=a+1')
    print(b)

14


In [11]:
test()

14


## Parsing and  Analyzing Python Source

In [14]:
#We all know that we can execute a python statement in a string form if supplied to a exec statement


x = 42

eval('x**2+3*4+5')


exec('for i in range(10): print(i)')

0
1
2
3
4
5
6
7
8
9


## Disassembling Python Byte Code

In [15]:
# You want to know in detail what your code is doing under the covers by disassembling it into lower-level byte code used by the interpreter

In [16]:
#The dis module can be used to output a disassembly of any python function. For example


def countdown(n):
    while n>0:
        print('T-minus', n)
        n -= 1
    print('Blastoff!')

import dis
dis.dis(countdown)


  5           0 LOAD_FAST                0 (n)
              2 LOAD_CONST               1 (0)
              4 COMPARE_OP               4 (>)
              6 POP_JUMP_IF_FALSE       17 (to 34)

  6     >>    8 LOAD_GLOBAL              0 (print)
             10 LOAD_CONST               2 ('T-minus')
             12 LOAD_FAST                0 (n)
             14 CALL_FUNCTION            2
             16 POP_TOP

  7          18 LOAD_FAST                0 (n)
             20 LOAD_CONST               3 (1)
             22 INPLACE_SUBTRACT
             24 STORE_FAST               0 (n)

  5          26 LOAD_FAST                0 (n)
             28 LOAD_CONST               1 (0)
             30 COMPARE_OP               4 (>)
             32 POP_JUMP_IF_TRUE         4 (to 8)

  8     >>   34 LOAD_GLOBAL              0 (print)
             36 LOAD_CONST               4 ('Blastoff!')
             38 CALL_FUNCTION            1
             40 POP_TOP
             42 LOAD_CONST               0 (

In [17]:
# The raw byte code interpreted by the dis() funciton is available on functions as follows

countdown.__code__.co_code

b'|\x00d\x01k\x04r\x11t\x00d\x02|\x00\x83\x02\x01\x00|\x00d\x038\x00}\x00|\x00d\x01k\x04s\x04t\x00d\x04\x83\x01\x01\x00d\x00S\x00'

In [20]:
def sub(x,y):
    return x-y
sub(3,4)
sub.__code__.co_code
#Using co-code one can extract the byte code of the function




b'|\x00|\x01\x18\x00S\x00'